# Querying

This notebook demonstrates how to retrieve, query and search data using the Forge.

## Initialisation

Run the [Blue Brain Nexus project creation notebook](./00%20-%20Nexus_Project_Initialisation.ipynb) to create a Blue Brain Nexus project if you don't have one.

In [ ]:
!pip install git+https://github.com/BlueBrain/nexus-forge

In [ ]:
import getpass

The [Nexus web application](https://sandbox.bluebrainnexus.io/web) can be used to login and get a token.

- Step 1: From the opened web page, click on the login button on the right corner and follow the instructions.

![login-ui](https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/login-ui.png)

- Step 2: At the end you’ll see a token button on the right corner. Click on it to copy the token.

![login-ui](https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/copy-token.png)


In [ ]:
token = getpass.getpass()

In [ ]:
from kgforge.core import KnowledgeGraphForge

In [ ]:
# Clone the repository if in Google Colab
import os 

!pwd
tutorial_base_dir = "./nexus-forge"
if os.path.exists(tutorial_base_dir):
  !rm -Rf $tutorial_base_dir

!git clone --single-branch https://github.com/BlueBrain/nexus-forge.git


os.chdir("/".join([tutorial_base_dir,"examples/notebooks/nexus-demo"]))

print("The working directory is now:")
!pwd

In [ ]:
#Let get some SHACL shapes from https://github.com/INCF/neuroshapes.git
import os 

neuroshapes_dir = "./neuroshapes"
if os.path.exists(neuroshapes_dir):
  !rm -Rf $neuroshapes_dir
! git clone https://github.com/INCF/neuroshapes.git
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/dataset" "./neuroshapes/shapes/neurosciencegraph/commons/" 
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/activity" "./neuroshapes/shapes/neurosciencegraph/commons/" 
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/entity" "./neuroshapes/shapes/neurosciencegraph/commons/" 
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/ontology" "./neuroshapes/shapes/neurosciencegraph/commons/" 
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/person" "./neuroshapes/shapes/neurosciencegraph/commons/" 

In [ ]:
# Set up some configurations

org ="tutorialnexus"
project ="myProject"
bucket = org+"/"+project
endpoint = "https://sandbox.bluebrainnexus.io/v1"


config = {
  "Model": {
    "name": "RdfModel",
    "origin": "directory",
    "source": "./neuroshapes/shapes/neurosciencegraph/commons/",
    "context": {
      "iri": "./neuroshapes_context.json"
    }
  },
  "Store": {
    "name": "BlueBrainNexus",
    "endpoint": "https://sandbox.bluebrainnexus.io/v1",
    "versioned_id_template": "{x.id}?rev={x._store_metadata._rev}",
    "file_resource_mapping": "../../configurations/nexus-store/file-to-resource-mapping.hjson"
  },
  "Formatters": {
    "identifier": "https://kg.example.ch/{}/{}"
  }
}


In [ ]:
# Get a KnowledgeGraphForge session

forge = KnowledgeGraphForge(config, endpoint=endpoint,bucket=bucket, token=token)

## Imports

In [ ]:
from kgforge.core import Resource

## Retrieval

In [ ]:
jane = Resource(type="Person", name="Jane Doe", email="jane.doe@sxample.org")

In [ ]:
forge.register(jane)

In [ ]:
jane.id

In [ ]:
resource = forge.retrieve(jane.id)

In [ ]:
resource == jane

### specific version

In [ ]:
jane = Resource(type="Person", name="Jane Doe")

In [ ]:
forge.register(jane)

In [ ]:
forge.tag(jane, "v1")

In [ ]:
jane.email = "jane.doe@epfl.ch"

In [ ]:
forge.update(jane)

In [ ]:
jane._store_metadata._rev

In [ ]:
jane_v1 = forge.retrieve(jane.id, version=1)

In [ ]:
jane_v1_tag = forge.retrieve(jane.id, version="v1")

In [ ]:
jane_v1 == jane_v1_tag

In [ ]:
forge.as_json(jane_v1)

In [ ]:
forge.as_json(jane)

### error handling

In [ ]:
resource = forge.retrieve("123")

In [ ]:
print(resource)

## Searching

In [ ]:
jane = Resource(type="Person", name="Jane Doe")

In [ ]:
john = Resource(type="Person", name="John Smith")

In [ ]:
association_jane = Resource(type="Dataset", contribution=jane)

In [ ]:
association_john = Resource(type="Dataset", contribution=john)

In [ ]:
associations = [association_jane, association_john]

In [ ]:
forge.types()

In [ ]:
forge.register(associations)

`Association` is a known type by the Model, and the `paths` method will load the data structure for the given type. Refer to the `11 - Modeling.ipynb` notebook to learn about Modeling and Types.

In [ ]:
p = forge.paths("Dataset")

You have autocompletion on `p` and this can be used to build a search.

In [ ]:
resources = forge.search(p.type.id == "Person", limit=5)

In [ ]:
type(resources)

In [ ]:
len(resources)  

In [ ]:
type(resources[0])

In [ ]:
forge.as_dataframe(resources)

In [ ]:
forge.as_dataframe(resources, store_metadata=True)

### nested field querying

You have autocompletion on `p` and also on nested properties like `p.agent`.

In [ ]:
resources = forge.search(p.type.id == "Dataset", p.contribution.type == "Person", limit=5)

In [ ]:
len(resources)

In [ ]:
forge.as_dataframe(resources)

## Graph traversing

SPARQL is used to traverse the graph. The user can provide simplified queries: no prefixes, no compacted URIs, nor full URIs is required, only the term (property) available in the Model to the user.  Refer to the `11 - Modeling.ipynb` notebook to learn about Templates.

In [ ]:
jane = Resource(type="Person", name="Jane Doe")

In [ ]:
john = Resource(type="Person", name="John Smith")

In [ ]:
association = Resource(type="Association", agent=[jane, john])

In [ ]:
forge.register(association)

In [ ]:
forge.template("Dataset",only_required=True)

In [ ]:
query = """
    SELECT ?x ?name
    WHERE {
        ?x type Dataset ;
           name ?name .
    }
"""

In [ ]:
resources = forge.sparql(query, limit=5)

In [ ]:
type(resources)

In [ ]:
len(resources)

In [ ]:
type(resources[0])

### rewritten query display

In [ ]:
resources = forge.sparql(query, debug=True)

## Downloading

In [ ]:
jane = Resource(type="Person", name="Jane Doe")

In [ ]:
! ls -p ../../data | egrep -v /$

In [ ]:
distribution = forge.attach("../../data")

In [ ]:
association = Resource(type="Association", agent=jane, distribution=distribution)

In [ ]:
forge.register(association)

In [ ]:
forge.download(association, "distribution.contentUrl", "./downloaded/")

In [ ]:
! ls ./downloaded/

In [ ]:
! rm -R ./downloaded/